# Imports necessary packages and predefined values

In [ ]:
import galois # Need to install galois package: pip install galois
import numpy as np
from convert_alist import readAlist 

## [Option 1] The following are the parameters of the self-dual code included in the GO03_self_dual/ directory
alistDirPath = "./alistMats/GO03_self_dual/"
length_dist_dict = {4:2, 6:2, 8:2, 10:2, 12:4, 14:4, 16:4, 18:4, 20:4, 22:6, 24:8, 26:6, 28:6, 30:6, 32:8, 34:6, 36:8, 38:8, 40:8, 42:8, 44:8, 46:8, 48:8, 50:8, 52:10, 54:8, 56:10, 58:10, 60:12, 62:10, 64:10}
alistEnd = ".alist"
if_self_dual = True

# ### [Option 2] The following are the parameters of the *dual-containing* [n, (n+1)//2, d] QR code directory QR_dual_containing/
# alistDirPath = "./alistMats/QR_dual_containing/" 
# length_dist_dict = {7:3, 17:5, 23:7, 47:11, 79:15, 103:19, 167:23} 
# alistEnd = ".alist"
# if_self_dual = False

# Import parity check matrices of classical self-dual/dual-containing codes from alist files
The self-dual codes are contructed based on [[GO03]](https://www.sciencedirect.com/science/article/pii/S107157970300011X).

The dual-containing codes are Quadratic Residue codes, whose length is a prime number.
<!-- [1] P. Gaborit and A. Otmani, “Experimental constructions of self-dual codes,” Finite Fields and Their Applications, vol. 9, no. 3, pp. 372–394, July 2003. -->
 

## Vary the $n$ for a different length


In [ ]:
n = 24 # all even lengths from 4 to 64
d = length_dist_dict[n]
F2 = galois.GF(2)
alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + alistEnd

GenMat = F2(readAlist(alistFilePath))
if if_self_dual:
    print("Generator matrix of the [n=%s,k=%s,d=%s] self-dual code:" % (n, n//2, d))
else:
    print("Generator matrix of the [n=%s,k=%s,d=%s] dual-containing code:" % (GenMat.shape[1], GenMat.shape[0], d))
print(GenMat)
print("Is self-dual? ", GenMat @ GenMat.transpose() )

Generator matrix of the [n=24,k=12,d=6] self-dual code:
[[0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1]
 [0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1]]
Is self-orthogonal?  [[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0

# Puncturing of a column to create a self-dual CSS code
The code is constructed according to Fig.2 in [[JA25]](https://ieeexplore.ieee.org/abstract/document/11006024).

In [3]:
if if_self_dual:
    G_punctured = GenMat[:, :-1]  # Puncture the last column
    print("Punctured code: [n=%s,k=%s]" % (G_punctured.shape[1], G_punctured.shape[0]))
    H_punctured = G_punctured.null_space() # Parity-check matrix of the punctured code = generator matrix of the dual of the punctured code
    print("Dual of the punctured code: [n=%s,k=%s]" % (H_punctured.shape[1], H_punctured.shape[0]))
    G_punctured @ H_punctured.T == 0  # Verify orthogonality
    #generate a H_CSS with H_punctured being in the left-upper and right-lower blocks and 0s elsewhere
    H_CSS = np.block([
        [H_punctured, np.zeros_like(H_punctured)],
        [np.zeros_like(H_punctured), H_punctured]
    ]); 
    print("The sympletic matrix of the self-dual CSS code [[n=%s,k=%s, d>=%s]]:\n" % (H_punctured.shape[1], H_punctured.shape[1]-2*H_punctured.shape[0], d-1))
else:
    H_punctured = GenMat.null_space()  # For GenMat being a dual-containing code, directly use the null-space of the generator matrix directly as the parity-check matrix
    #generate a H_CSS with H_punctured being in the left-upper and right-lower blocks and 0s elsewhere
    H_CSS = np.block([
        [H_punctured, np.zeros_like(H_punctured)],
        [np.zeros_like(H_punctured), H_punctured]
    ]);
    print("The sympletic matrix of the self-dual CSS code [[n=%s,k=%s, d>=%s]]:\n" % (H_punctured.shape[1], H_punctured.shape[1]-2*H_punctured.shape[0], d))
print(H_CSS) 

Punctured code: [n=23,k=12]
Dual of the punctured code: [n=23,k=11]
The sympletic matrix of the self-dual CSS code [[n=23,k=1, d>=5]]:

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]
